<h1>Copulas: Synthetizing the Insurance Dataset - Module 2</h1>
<p>
Here we create a separate copula for each group. We then look at performance metrics to assess the quality of synthetic data, comparing the single-copula method with a customized copula for each group. I then discuss in more details parametric copulas, when empirical quantiles are replaced by a parametric distribution, with parameters estimated on the real data. The content is as follows:
<ol>
    <li><a href="#section1">Imports and reading the data</a>
    <li><a href="#section2">Creating the groups, copula for each group, and synthetic data</a>
    <li><a href="#section3">Doing the same as in section 2, but without grouping</a>
    <li><a href="#section4">Assessing quality of synthetized data</a>
    <li><a href="#section5">Alternatives to Hellinger distance</a>
    <li><a href="#section6">Parametric copula</a>
</ol>

<a id='section1'></a>
<h2>1. Imports and reading the data</h2>
<p>
The data is stored as a csv file in the local directory. It has the following features: age, sex, bmi, children, smoker, region, charges.

In [1]:
import pandas as pd
from scipy.stats import norm
import numpy as np

In [2]:
# source: https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset
# Fields: age, sex, bmi, children, smoker, region, charges
    
url="https://raw.githubusercontent.com/VincentGranville/Main/main/insurance.csv"
# make sure fields don't contain commas
data = pd.read_csv(url)
print(data.head(10))

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520
5   31  female  25.740         0     no  southeast   3756.62160
6   46  female  33.440         1     no  southeast   8240.58960
7   37  female  27.740         3     no  northwest   7281.50560
8   37    male  29.830         2     no  northeast   6406.41070
9   60  female  25.840         0     no  northwest  28923.13692


<a id='section2'></a>
<h2>2. Creating the groups, copula for each group, and synthetic data</h2>
<p>
First we create the group structure. The hash table <code>groupReal</code> containing the real data (organized by group) is indexed by key = (group, idx) where 
<ul>
<li>idx is the index (number) of an observation in the "group" in question. Each observation is a list consisting of 4 values: age, bmi, number of children and charges. Thus an observation is the value in the key-value pair that defines the hash table.
<li>group is a tab-separated index consisting of gender (male/female), smoking status (yes/no) and region (NorthEast, NorthWest, SouthEast, SouthWest).
</ul>
The <code>groupCount</code> hash table stores the number of observations in each group. Later on, 
 in the function <code>gaussian_to_synth</code>, I create the hash table
<code>groupSynth</code> that contains the synthetic data. 

In [3]:
groupCount = {}
groupReal = {}
for k in range(0, len(data)):  
    obs = data.iloc[k]   # get observation number k
    group = obs[1] +"\t"+obs[4]+"\t"+obs[5]
    if group in groupCount:
        cnt = groupCount[group]
        groupReal[(group,cnt)]=(obs[0],obs[2],obs[3],obs[6]) 
        groupCount[group] += 1    
    else:
        groupReal[(group,0)]=(obs[0],obs[2],obs[3],obs[6]) 
        groupCount[group] = 1

In [4]:
for group in groupCount:
    print(group, groupCount[group])

female	yes	southwest 21
male	no	southeast 134
male	no	northwest 132
female	no	southeast 139
female	no	northwest 135
male	no	northeast 125
female	yes	southeast 36
male	no	southwest 126
male	yes	southeast 55
female	no	northeast 132
male	yes	southwest 37
female	no	southwest 141
female	yes	northeast 29
male	yes	northeast 38
male	yes	northwest 29
female	yes	northwest 29


In [5]:
print(groupReal[("female\tyes\tsouthwest",0)])
print(groupReal[("female\tyes\tsouthwest",1)])
print(groupReal[("female\tyes\tsouthwest",2)])
print(groupReal[("female\tyes\tsouthwest",3)])
print(groupReal[("female\tyes\tsouthwest",20)])

(19, 27.9, 0, 16884.924)
(37, 34.8, 2, 39836.519)
(64, 31.3, 2, 47291.055)
(19, 28.3, 0, 17081.08)
(19, 34.7, 2, 36397.576)


In [6]:
def create_table(group, groupCount, groupReal):

    # extract data corresponding to specific group, from big table groupReal

    nobs = groupCount[group]
    age = []
    bmi = []
    children = []
    charges = []
    for cnt in range(nobs):
        features = groupReal[(group,cnt)]
        age.append(float(features[0]))       # uniform outside very young or very old
        bmi.append(float(features[1]))       # Gaussian distribution?
        children.append(float(features[2]))  # geometric distribution?
        charges.append(float(features[3]))   # bimodal, not gaussian 
        real = np.stack((age, bmi, children, charges), axis = 0)
    return(real)

In [7]:
def gaussian_to_synth(real, gfg, group, nobs_synth, groupSynth):

    # turn multivariate gaussian gfg into synth. data, update groupSynth 
    # this is done for a specific group, creating nobs_synth obs.

    age = real[0,:]
    bmi = real[1,:]
    children = real[2,:]
    charges = real[3,:]

    g_age = gfg[:,0]
    g_bmi = gfg[:,1]
    g_children = gfg[:,2]
    g_charges = gfg[:,3]

    for k in range(nobs_synth):   

        u_age = norm.cdf(g_age[k])                     # u stands for uniform[0, 1]
        u_bmi = norm.cdf(g_bmi[k])
        u_children = norm.cdf(g_children[k])
        u_charges = norm.cdf(g_charges[k])

        s_age = np.quantile(age, u_age)                # synthesized age 
        s_bmi = np.quantile(bmi, u_bmi)                # synthesized bmi
        s_children = np.quantile(children, u_children) # synthesized children
        s_charges = np.quantile(charges, u_charges)    # synthesized charges

        # add k-th synth. obs. for group in question, to groupSynth
        groupSynth[(group, k)] = [s_age, s_bmi, s_children, s_charges] 

    return()

The parameter <code>seed</code> below allows for replicability. Also the quality of the synthetic data may depend on the seed.

In [8]:
seed = 453
np.random.seed(seed)
groupSynth = {}

This is the core of the algorithm, with a loop over each group. The final synthetic data <code>groupSynth</code> is updated in this loop, one group at a time. The array <code>real</code> is a temporary table containing the real data for the group in question, in a simple format to easily compute the correlation matrix <code>corr</code> for the group in question. This matrix is used to produce the Gaussian multivariate vector which is used in turn to produce the synthetic data. The number of synthetic observations produced for each group is <code>nobs_synth</code>. Here, it matches the corresponding number in the real data. 

In [9]:
for group in groupCount:

    real = create_table(group, groupCount, groupReal) 
    n_var = real.shape[0] 
    zero = np.zeros(n_var) 
    corr = np.corrcoef(real)       # correlation matrix for Gaussian copula for this group
    nobs_synth = groupCount[group] # number of synthetic obs to create for this group
    gfg = np.random.multivariate_normal(zero, corr, nobs_synth) 
    gaussian_to_synth(real, gfg, group, nobs_synth, groupSynth)

In [10]:
for group, k in groupSynth:

    obs = groupSynth[(group,k)] # this is k-th synth. obs. for group in question

    # print synth. data for sample group: age, bmi, children, charges
    if group == "female\tyes\tsouthwest":
        print("%6.2f %7.2f %6.2f %10.2f" % (obs[0], obs[1], obs[2], obs[3]))

 29.24   31.05   2.00   31812.65
 37.79   27.89   2.00   27595.47
 36.72   26.53   1.00   18972.35
 19.25   21.83   2.00   16925.74
 19.00   30.78   0.00   17545.09
 46.02   30.97   2.00   26974.52
 19.00   27.18   0.00   17380.77
 29.27   26.80   0.00   18936.03
 48.07   31.40   1.00   40524.87
 19.85   21.70   1.18   14858.06
 19.00   28.15   1.25   17297.46
 36.06   31.56   0.00   34401.89
 19.00   35.93   0.00   45531.59
 46.72   33.87   2.00   42533.74
 45.13   33.82   0.00   47295.20
 63.04   29.14   0.98   47258.66
 38.82   34.66   0.00   47673.08
 20.20   22.19   0.00   16970.50
 26.66   22.27   2.00   19484.48
 23.09   27.62   0.39   18924.28
 39.16   31.01   2.00   34694.34


<a id='section3'></a>
<h2>3. Doing the same as in section 2, but without grouping</h2>
<p>
The goal is to re-use the same coding architecture but this time assuming there is one single group that contains all the data. The purpose is to compare performance metrics (quality of synthetization) obtained via grouping (a separate copula for each group), versus no grouping (one global copula).
<p>
    New main tables are produced: <code>nogroupSynth</code>, <code>nogroupReal</code> and <code>nogroupCount</code>. They are the equivalent of <code>groupSynth</code>, <code>groupReal</code> and 
    <code>groupCount</code> used when working with actual groups. They represent respectively the synthesized data, the real data, and the group structure. Since there is no group - or in other words a single global group - iterations over the groups are reduced to just one iteration, making the loop useless.  The "by group" loops are kept only for compatibility with the "multiple groups" case. 

Now creating the single-group structure.

In [11]:
nogroupCount = {}
nogroupReal = {}
for k in range(0, len(data)):  
    obs = data.iloc[k]   # get observation number k
    group = "nogroup"
    if group in nogroupCount:
        cnt = nogroupCount[group]
        nogroupReal[(group,cnt)]=(obs[0],obs[2],obs[3],obs[6]) 
        nogroupCount[group] += 1    
    else:
        nogroupReal[(group,0)]=(obs[0],obs[2],obs[3],obs[6]) 
        nogroupCount[group] = 1

Producing the synthetized data <code>nogroupSynth</code>.

In [12]:
nogroupSynth = {}

for group in nogroupCount:

    real = create_table(group, nogroupCount, nogroupReal) 
    n_var = real.shape[0] 
    zero = np.zeros(n_var) 
    corr = np.corrcoef(real)       # correlation matrix for Gaussian copula for this group
    nobs_synth = nogroupCount[group] # number of synthetic obs to create for this group
    gfg = np.random.multivariate_normal(zero, corr, nobs_synth) 
    gaussian_to_synth(real, gfg, group, nobs_synth, nogroupSynth)

Showing the first 10 synthetized observations, with features age, bmi, children, charges.

In [13]:
for group, k in nogroupSynth:

    obs = nogroupSynth[(group,k)] # this is k-th synth. obs. for group in question

    # print synth. data for sample group: age, bmi, children, charges
    if k < 10:
        print("%6.2f %7.2f %6.2f %10.2f" % (obs[0], obs[1], obs[2], obs[3]))

 59.00   33.24   2.00   16096.82
 34.00   28.12   3.00    9871.41
 61.00   25.46   0.00   42734.81
 39.00   30.69   1.00   26122.09
 50.00   20.09   3.00   11337.16
 62.00   35.09   3.00   45705.89
 27.00   25.08   1.00    1907.57
 27.00   31.02   2.00    5255.97
 59.00   29.80   3.00   11553.41
 58.00   26.50   0.00   21775.68


<a id='section4'></a>
<h2>4. Assessing quality of synthetized data</h2>
<p>
In particular, comparing grouping with no grouping. As an optional exercise, it would be interesting to compare the results obtained with various seeds. Or comparing grouping with no grouping by using the full set of features, using dummy variables for sex, smoking status and regions.
<p>
First, let's turn the data obtained so far into more traditional arrays for easy processing.

In [14]:
arr_nogroupSynth = []
arr_groupSynth = []
arr_groupReal = []

for group, k in nogroupSynth:
    obs = nogroupSynth[(group,k)]
    arr_nogroupSynth.append(obs)
arr_nogroupSynth = np.transpose(arr_nogroupSynth)
arr_nogroupSynth = np.asarray(arr_nogroupSynth, dtype = np.float64, order ='C')

for group, k in groupSynth:
    obs = groupSynth[(group,k)]
    arr_groupSynth.append(obs)
arr_groupSynth = np.transpose(arr_groupSynth)
arr_groupSynth = np.asarray(arr_groupSynth, dtype = np.float64, order ='C')

for group, k in groupReal:
    obs = groupReal[(group,k)]
    arr_groupReal.append(obs)
arr_groupReal = np.transpose(arr_groupReal)
arr_groupReal = np.asarray(arr_groupReal, dtype = np.float64, order ='C')

Now let's compute correlation matrices and some averages. 

In [15]:
corr1 = np.corrcoef(real) 
corr2 = np.corrcoef(arr_groupReal)
corr3 = np.corrcoef(arr_groupSynth)
corr4 = np.corrcoef(arr_nogroupSynth)

print("\nCorr matrix real:\n",corr1)
print("\nCorr matrix real group:\n",corr2)
print("\nCorr matrix synth. group:\n",corr3)
print("\nCorr matrix synth. nogroup:\n",corr3)


Corr matrix real:
 [[1.         0.10927188 0.042469   0.29900819]
 [0.10927188 1.         0.0127589  0.19834097]
 [0.042469   0.0127589  1.         0.06799823]
 [0.29900819 0.19834097 0.06799823 1.        ]]

Corr matrix real group:
 [[1.         0.10927188 0.042469   0.29900819]
 [0.10927188 1.         0.0127589  0.19834097]
 [0.042469   0.0127589  1.         0.06799823]
 [0.29900819 0.19834097 0.06799823 1.        ]]

Corr matrix synth. group:
 [[1.         0.10394322 0.03567531 0.22895421]
 [0.10394322 1.         0.00757671 0.19321724]
 [0.03567531 0.00757671 1.         0.05468817]
 [0.22895421 0.19321724 0.05468817 1.        ]]

Corr matrix synth. nogroup:
 [[1.         0.10394322 0.03567531 0.22895421]
 [0.10394322 1.         0.00757671 0.19321724]
 [0.03567531 0.00757671 1.         0.05468817]
 [0.22895421 0.19321724 0.05468817 1.        ]]


The first two data sets, <code>real</code> and <code>arr_groupReal</code> are of course identical, thus the correlation matrix are identical. The last two, <code>arr_groupSynth</code> and <code>arr_nogroupSynth</code> are different, yet lead to the same correlation matrix. The fit between real and synthetic is pretty good as far a replicating the correlation structure is concerned. This is true whether grouping is involved or not.

In [16]:
print("\nmean real: [age, bmi, children, charges]\n",np.mean(arr_groupReal, axis=1))
print("\nmean synth. group: [age, bmi, children, charges]\n",np.mean(arr_groupSynth, axis=1))
print("\nmean synth. nogroup: [age, bmi, children, charges]\n",np.mean(arr_nogroupSynth, axis=1))


mean real: [age, bmi, children, charges]
 [3.92070254e+01 3.06633969e+01 1.09491779e+00 1.32704223e+04]

mean synth. group: [age, bmi, children, charges]
 [3.88389577e+01 3.10462786e+01 1.03336422e+00 1.30465876e+04]

mean synth. nogroup: [age, bmi, children, charges]
 [3.91705999e+01 3.04771953e+01 1.11876599e+00 1.31274218e+04]


Grouping does not lead to improvements over no grouping, regarding global statistics. Clearly, the two datasets "group" and "nogroup" show differences now; "nogroup" looks a tiny bit closer to the real data, but this could have happened by chance.

Now let's look at the details for a specific "group", to see how group significantly outperforms "nogroup". Again, 
the first step is to turn the data into arrays that are easy to handle.

In [17]:
group_MNS = "male\tyes\tsoutheast"
nobs_MNS = groupCount[group_MNS]
print("\nObs. in group Male/Smoker/SouthEast:",nobs_MNS)
arr_group_MNS_synth = []
arr_group_MNS_real = []


Obs. in group Male/Smoker/SouthEast: 55


In [18]:
for k in range(nobs_MNS):
    obs_s = groupSynth[(group_MNS,k)]
    obs_r = groupReal[(group_MNS,k)]
    arr_group_MNS_synth.append(obs_s)
    arr_group_MNS_real.append(obs_r)

In [19]:
arr_group_MNS_synth = np.transpose(arr_group_MNS_synth)
arr_group_MNS_synth = np.asarray(arr_group_MNS_synth, dtype = np.float64, order ='C')
arr_group_MNS_real = np.transpose(arr_group_MNS_real)
arr_group_MNS_real = np.asarray(arr_group_MNS_real, dtype = np.float64, order ='C')

In [20]:
print("\nmean real group MNS: [age, bmi, children, charges]\n",np.mean(arr_group_MNS_real, axis=1))
print("\nmean synth. group MNS: [age, bmi, children, charges]\n",np.mean(arr_group_MNS_synth, axis=1))


mean real group MNS: [age, bmi, children, charges]
 [4.00545455e+01 3.36500000e+01 1.03636364e+00 3.60298394e+04]

mean synth. group MNS: [age, bmi, children, charges]
 [3.72394058e+01 3.61261220e+01 9.84307691e-01 3.76027507e+04]


Without grouping, the average charges for that froup would be the average charges for the whole population, that is around \\$13,000. With grouping (and thus with a customized copula for that group), the average charges for that group, in the synthetic data, is \\$37,602. In the real data, it is \\$36,029. Pretty close! But very different from the global average, by an order of magnitude. That group has 55 observations, enough to get decent synthetization given the small number of features (4 features).

<a id='section5'></a>
<h2>5. Alternatives to Hellinger distance</h2>
<p>
Table-Evaluator is a Python library offering many possibilities. In Module 3, I will discuss a correlation matrix distance. Here I show how to use the Kolmogorov distance. The Hellinger distance requires to compute the empirical probability density functions (EPDF) attached to vectors of observations (features). It works great with discrete distributions (age, number of children), but it is subject to arbitrary parameters when dealing with continuous distributions (bmi, charges) unless you bin them to make them discrete. The ECDF (empirical cumulative distribution function) solves this problem and is more widespread in Python. The Kolmogorov distance is based on the ECDF. Note that it looks at individual features, not at the inter-dependencies. 
    
The Hellinger distance in theory is easy to extend to multivariate features (thus handling the dependencies structure), but in practice this desirable property is ignored and many practitioners stick to comparing single rather than joint features. In that case, why not use Kolmogorov instead? Note that a full-fledge implementation of Hellinger for joint features comes with its own problems: a large number of multivariate bins, many too small and causing problems unless aggregated. We avoid all of this here, focusing on single features one at a time. We capture the dependencies via the correlation matrix instead, which is a good first order (linear) approximation. 

In [21]:
from scipy.stats import ks_2samp

nvar = arr_nogroupSynth.shape[0]  # number of features
print("KS distance between real and synth, for each feature (no grouping):")
for idx in range(nvar):
    test = ks_2samp(arr_nogroupSynth[idx,:], arr_groupReal[idx,:])
    print("feature %2d: KS distance = %5.4f  P-value = %5.4f" 
        %(idx, test.statistic, test.pvalue))

KS distance between real and synth, for each feature (no grouping):
feature  0: KS distance = 0.0112  P-value = 1.0000
feature  1: KS distance = 0.0172  P-value = 0.9891
feature  2: KS distance = 0.0164  P-value = 0.9936
feature  3: KS distance = 0.0299  P-value = 0.5884


In [22]:
nvar = arr_groupSynth.shape[0]  # number of features
print("KS distance between real and synth, for each feature (grouping):")
for idx in range(nvar):
    test = ks_2samp(arr_groupSynth[idx,:], arr_groupReal[idx,:])
    print("feature %2d: KS distance = %5.4f  P-value = %5.4f" 
        %(idx, test.statistic, test.pvalue))

KS distance between real and synth, for each feature (grouping):
feature  0: KS distance = 0.0306  P-value = 0.5564
feature  1: KS distance = 0.0419  P-value = 0.1918
feature  2: KS distance = 0.0254  P-value = 0.7809
feature  3: KS distance = 0.0321  P-value = 0.4944


The Kolmogorov-Smirnov distance (0 = perfect match, 1 = worst) is really low for each feature, confirming the goodness of fit between real and synthetic data. This is further confirmed by the high P-values: a low value (close to 0) would suggest that the two distributions compared in <code>ks_2samp</code> are different. If you want a single metric, use the maximum of these 4 distances. See how it varies when using a different seed. Note that features 0, 1, 2, 3 are respectively age, bmi, number of children, and charges.

In [23]:
nvar = arr_group_MNS_synth.shape[0]  # number of features
print("KS distance between real and synth for group MNS, for each feature:")
for idx in range(nvar):
    test = ks_2samp(arr_group_MNS_synth[idx,:], arr_group_MNS_real[idx,:])
    print("feature %2d: KS distance = %5.4f  P-value = %5.4f" 
        %(idx, test.statistic, test.pvalue))

KS distance between real and synth for group MNS, for each feature:
feature  0: KS distance = 0.1455  P-value = 0.6102
feature  1: KS distance = 0.1636  P-value = 0.4565
feature  2: KS distance = 0.0909  P-value = 0.9789
feature  3: KS distance = 0.1091  P-value = 0.9031


Same test, but this type just for one group: MNS. Again, great fit (even better!) based on P-values. The distances are a bit larger because the sample size is much smaller. 

Now looking at additional stats: minima and maxima for each feature, real vs synthetic (group / nogroup).

In [24]:
print("\nMinimum: [age bmi children charges]")
print("nogroup:",np.min(arr_nogroupSynth, axis = 1))
print("group  :",np.min(arr_groupSynth, axis = 1))
print("real   :",np.min(arr_groupReal, axis = 1))


Minimum: [age bmi children charges]
nogroup: [  18.           16.86776217    0.         1123.38907911]
group  : [  18.           16.0368089     0.         1128.62393323]
real   : [  18.       15.96      0.     1121.8739]


In [25]:
print("\nMaximum: [age bmi children charges]")
print("nogroup:",np.max(arr_nogroupSynth, axis = 1))
print("group  :",np.max(arr_groupSynth, axis = 1))
print("real   :",np.max(arr_groupReal, axis = 1))


Maximum: [age bmi children charges]
nogroup: [6.40000000e+01 5.29651097e+01 5.00000000e+00 5.81479332e+04]
group  : [6.40000000e+01 5.24877257e+01 5.00000000e+00 6.11376411e+04]
real   : [6.4000000e+01 5.3130000e+01 5.0000000e+00 6.3770428e+04]


The method based on nogroup underestimates the maximum charge: \\$58,147 (no group) versus \\$61,137 (grouping) versus \\$63,770 (real). It also overestimates the minimum bmi: 16.86 (no group) versus 16.03 (grouping) versus 15.96 (real).

<a id='section6'></a>
<h2>6. Parametric copula</h2>
<p>
In Module 1, I discussed how we could use a Geometric distribution of parameter $p$ instead of the empirical distribution (actually the empirical quantiles) for the number of children. You estimate $p$ on the real data. However, it leads to a less than ideal distribution, the maximum number of children being frequently over 10, despite the mean number being correct. In the real dataset, the maximim is 5.
    <p>
To overcome this problem, one can use a generalized geometric distribution with 2 parameters. I show how to do it in my article
  <a href="https://mltechniques.com/2023/03/30/smart-grid-search-case-study-with-hybrid-zeta-geometric-distributions-and-synthetic-data/">Smart Grid Search for Faster Hyperparameter Tuning</a>. It involves creating such a distribution, then estimating its parameters on the real data. The latter can be done using gradient descent. In my article, I use smart grid search instead. The Python implementation is in the last section in Module 1.